Import necessary libraries:

In [1]:
import pandas as pd
import numpy as np
import re
import string
from difflib import get_close_matches, SequenceMatcher
#!pip install python-Levenshtein
from fuzzywuzzy import fuzz

C:\Users\robin\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Import data:

In [2]:
houses = pd.read_csv('D:/Data/JacksonvilleHouses.csv')
print(houses.shape)
houses.head()

C:\Users\robin\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(252718, 97)


,RE,section,township,range,tile,mailing_address_1,mailing_address_2,mail_city,mail_state,mail_zipcode,...,street_type,unit,city,zipcode,max_price,max_date,most_recent_date,most_recent_price,q_sales,q_sales_3yrs
0,0000060030R,1,2S,23E,3401.0,2503 SUMMERFIELD LN,NaN,JACKSONVILLE,FL,32234,...,LN,NaN,JACKSONVILLE,32234,192000.0,2004-07-09,2004-07-09,192000.0,1.0,NaN
1,0000070010R,1,2S,23E,3401.0,C/O TAX DEPARTMENT,9540 SAN JOSE BLVD,JACKSONVILLE,FL,32257,...,HWY,NaN,JACKSONVILLE,32234,150000.0,2018-01-04,2018-01-04,150000.0,2.0,1.0
2,0000090100R,12,2S,23E,3412.0,2495 U S 301 HWY N,NaN,BALDWIN,FL,32234,...,HWY,NaN,JACKSONVILLE,32234,NaN,NaN,NaN,NaN,NaN,NaN
3,0000110000R,12,2S,23E,3412.0,2425 US HWY 301 N,NaN,BALDWIN,FL,32234,...,HWY,NaN,JACKSONVILLE,32234,NaN,NaN,NaN,NaN,NaN,NaN
4,0000120010R,12,2S,23E,3412.0,2204 US HIGHWAY 301 N,NaN,JACKSONVILLE,FL,32234,...,HWY,NaN,JACKSONVILLE,32234,NaN,NaN,NaN,NaN,NaN,NaN


Remove spaces in column names:

In [3]:
new_cols = houses.columns.to_list()
new_cols = [x.replace(' ','_') for x in new_cols]
houses.rename(columns=dict(zip(houses.columns.to_list(),new_cols)), inplace=True)

Convert parcel IDs to same format as shapefiles for future visualizations:

In [4]:
houses['RE'] = houses['RE'].str.replace('R','')
houses['RE'] = [x[:6] + '-' + x[-4:] for x in houses['RE']]
houses['RE_b'] = houses['RE'] + '_' + houses['building'].map(str)

Combine section, township and range columns into a single column:

In [5]:
houses['section'] = houses['section'].astype('str')
houses['sec_township_range'] = houses['section'] + ' ' + houses['township'].map(str) + ' ' + \
        houses['range'].map(str)
houses.drop(columns=['section', 'township', 'range'], inplace=True)

Remove unneeded columns:

In [6]:
houses.drop(columns=['tile','style'], inplace=True)

Define functions to use in string cleaning:

In [7]:
def remove_punctuation(lst):
    """Removes punctuation from text strings.

    Parameters
    ----------
    lst : list
        List of strings

    Returns
    -------
    cleaned: list
        List of strings without punctuation
    """
    cleaned = []
    for thing in lst:
        for punctuation in string.punctuation:
            thing = thing.replace(punctuation,'')
        cleaned.append(thing)
    return cleaned

In [8]:
def remove_spacing(lst):
    """Removes consecutive spaces.

    Parameters
    ----------
    lst : list
        List of strings

    Returns
    -------
    cleaned: list
        List of strings without consecutive spaces
    """
    cleaned = []
    for thing in lst:
        thing = " ".join(thing.split())
        cleaned.append(thing)
    return cleaned

In [9]:
def replace_special(lst, repl_dictionary):
    """Replaces dictionary keys with dictionary values in strings.

    Parameters
    ----------
    lst : list
        List of strings
        
    repl_dictionary : dictionary
        Dictionary of replacements

    Returns
    -------
    cleaned: list
        List of strings with specified items replaced by specified values
    """
    cleaned = []        
    for thing in lst:
        for k in list(repl_dictionary.keys()):
            thing = thing.replace(k, repl_dictionary[k])
        cleaned.append(thing)
    return cleaned

In [10]:
def clean_strings(lst, repl_dictionary={}):
    """Capitalizes, removes beginning and ending spaces, removes punctuation, removes consecutive
    spaces and replaces specified substrings with specified values.

    Parameters
    ----------
    lst : list
        List of strings
        
    repl_dictionary : dictionary
        Dictionary of replacements

    Returns
    -------
    cleaned: list
        List of cleaned strings
    """
    lst = [x.upper().strip() for x in lst]
    lst = remove_punctuation(lst)
    lst = remove_spacing(lst)
    lst = replace_special(lst, repl_dictionary)
    return lst

Clean text columns:

In [11]:
houses['owner'] = clean_strings(houses['owner'])

In [12]:
### Remove 'care of' lines in mailing address columns:
houses['care_of'] = None

special = ['C/O 8052 MONCRIEF-DINSMORE RD','C/O 2811 AUBREY AVE','C/O ALAN CAIN USMTM TABUK UNIT 94114',
 'C/O 4006 STARRATT RD','C/O 4305 ROTH DR','C/O OFC WEALTH 7950 JONES BRANCH DR STE 700N']

houses.loc[houses['mailing_address_1'].isin(special)==True,'mailing_address_1'] = \
houses.loc[houses['mailing_address_1'].isin(special)==True,'mailing_address_1'].str.replace('C/O ','')

houses.loc[houses['mailing_address_1'].str.startswith('C/O')==True,'care_of'] = \
houses.loc[houses['mailing_address_1'].str.startswith('C/O')==True,'mailing_address_1'].str.replace('C/O ',\
    '').str.replace('TAX DEPARTMENT','').str.replace('DEPT','')

houses.loc[houses['mailing_address_1'].str.startswith('C/O')==True,'mailing_address_1'] = \
houses.loc[houses['mailing_address_1'].str.startswith('C/O')==True,'mailing_address_2']

houses.loc[houses['mailing_address_1']==houses['mailing_address_2'],'mailing_address_2'] = ''

houses.loc[houses['mailing_address_2'].str.startswith('C/O')==True,'care_of'] = \
houses.loc[houses['mailing_address_2'].str.startswith('C/O')==True,'mailing_address_2'].str.replace('C/O ',\
    '').str.replace('TAX DEPARTMENT','').str.replace('DEPT','')

houses.loc[houses['mailing_address_2'].str.startswith('C/O')==True,'mailing_address_2'] = ''

# for i in range(0,len(houses['mailing_address_1'])):
#     if houses['mailing_address_1'][i][:3] == 'C/O':
#         houses['mailing_address_1'][i] = str(houses['mailing_address_2'][i])
#         houses['mailing_address_2'][i] = ''

# houses['mailing_address_2'] = houses['mailing_address_2'].fillna('').astype('str')
# for i in range(0,len(houses['mailing_address_2'])):
#     if houses['mailing_address_2'][i][:3] == 'C/O':
#         houses['mailing_address_2'][i] = ''


In [13]:
address_dictionary = \
    {' U S ':' US ', 'P O ':'PO ', 'FLORIDA':'FL', 'STATE ROAD':'SR', 'STATE RD':'SR',
    ' FORT ':' FT ', ' SAINT':' ST', ' TRAIL':' TRL', ' PLACE':' PL', 'CIRCLE':'CIR', ' COURT':' CT', 
    ' BEACH':' BCH',
    ' LANE':' LN', 'STREET':'ST', 'AVENUE':'AVE', ' AV ':' AVE ', 'DRIVE':'DR', ' ROAD':' RD', 
    'HIGHWAY':'HWY',
    'BOULEVARD':'BLVD', ' CREEK':' CK', 'PLAZA':'PLZ',
    'NORTHEAST':'NE', 'NORTHWEST':'NW', 'SOUTHEAST':'SE', 'SOUTHWEST':'SW',
    'NORTH':'N', 'SOUTH':'S', 'EAST':'E', 'WEST':'W', 
    'FIRST':'1ST', 'SECOND':'2ND', 'THIRD':'3RD', 
    'FOURTH':'4TH', 'FIFTH':'5TH', 'SIXTH':'6TH', 'SEVENTH':'7TH', 'EIGHTH':'8TH', 'NINTH':'9TH', 
    'TENTH':'10TH', 'ELEVENTH':'11TH', 'TWELFTH':'12TH', 'THIRTEENTH':'13TH', 'FOURTEENTH':'14TH', 
    'FIFTEENTH':'15TH', 'SIXTEENTH':'16TH', 'SEVENTEENTH':'17TH', 'EIGHTEENTH':'18TH', 'NINETEENTH':'19TH',
    'UNIT':'', 'SUITE':'', ' STE ':' ', ' SET ':'', 'BUILDING':'', 'BLDNG':'', 'APARTMENT':'',
    ' APT ':''}

houses['mailing_address_1'] = clean_strings(houses['mailing_address_1'], address_dictionary)

In [14]:
houses['care_of'].value_counts()[:10]

INVITATION HOMES                      867
TRICON AMERICAN HOMES LLC             300
HAVENBROOK HOMES                      134
FIRST KEY HOMES LLC                    77
WRI PROPERTY MANAGEMENT LLC            68
MARK HENDERSON                         17
PREFERRED TRUST COMPANY                16
POINT TO POINT REALTY SAM HARRISON     13
GREEN CREEK ASSET MANAGEMENT LLC       13
JUDY OLEVNIK                           12
Name: care_of, dtype: int64

In [15]:
houses['mailing_address_2'] = houses['mailing_address_2'].fillna('')
houses['mailing_address_2'] = clean_strings(houses['mailing_address_2'], address_dictionary)
houses['mailing_address_2'] = ['' if x.startswith('PO ') else x for x in houses['mailing_address_2']]

### Combine mailing address columns
houses['mailing_address'] = houses['mailing_address_1'] + ' ' + houses['mailing_address_2']
houses['mailing_address'] = remove_spacing(houses['mailing_address'])

houses.drop(columns=['mailing_address_1', 'mailing_address_2'], inplace=True)

The mailing address city column is especially messy and thus needs extra steps to clean:

In [16]:
houses['mail_city'] = houses['mail_city'].fillna('NONE').astype('str')
### Remove numbers
houses['mail_city'] = [re.sub(r'[0-9]+', '', x) for x in houses['mail_city']]

### Fix certain misspellings and abbreviations
first_replacements = \
    {' ON ':' ONTARIO ', 'ONTARIO THE':'ON THE', ' GU ':' GUAM ', 'ISREAL':'ISRAEL', ' IL ':' ISRAEL ',
     'HONG KONG':'CHINA', ' CA ':' CANADA ', 'BCH':'BEACH', 'TOKYO':'JAPAN', 'LONDON':'UNITED KINGDOM',
     'TRINIDAD':'TRINIDAD AND TOBEGO', 'ZURICH':'SWITZERLAND', 'ONTARIO':'CANADA', 'TIKVA':'ISRAEL',
     'HAAYIN':'ISRAEL', 'TEL AVIV':'ISRAEL', 'KOREA':'SOUTH KOREA', 'MAIMI':'MIAMI'
    }

houses['mail_city'] = clean_strings(houses['mail_city'], first_replacements)

### Remove directions and standardize certain words
dirs = ['N', 'S', 'E', 'W', 'NORTH', 'SOUTH', 'EAST', 'WEST']
repl = {'IL':'ISRAEL', 'ST':'SAINT', 'FT':'FORT', 'MT':'MOUNT'}
cities = []
for city in houses['mail_city']:
    c = city.split()
    if c[0] in dirs:
        c = c[1:]
    cities.append(" ".join(c).strip())
cities2 = []
for city in cities:
    c = city.split()
    if c[0] in list(repl.keys()):
        c[0] = repl[c[0]]
    cities2.append(" ".join(c).strip())
houses['mail_city'] = cities2

### Fix two specific abbreviations
ny_index = houses[houses['mail_city'] == 'NEW YORK'].index.to_list()
for i in ny_index:
    houses['mail_city'][i] = 'NEW YORK'
jax_index = houses[houses['mail_city'] == 'JAX'].index.to_list()
for i in jax_index:
    houses['mail_city'][i] = 'JACKSONVILLE'

### If a foreign country is listed in the city column, change the value to 'FOREIGN COUNTRY'
country_list = [
    'SPAIN', 'ISRAEL', 'ETHIOPIA', 'CANADA', 'JAPAN', 'LEBANON','AUSTRALIA', 'COLUMBIA', 'BRAZIL', 'CHINA',
    'KOREA', 'TURKEY', 'BAHAMAS', 'VIETNAM', 'PERU', 'NEW ZEALAND', 'FRANCE', 'BERMUDA', 'SWITZERLAND',
    'TRINIDAD AND TOBEGO', 'ECUADOR', 'UNITED KINGDOM', 'CZECH REPUBLIC', 'JAMAICA']

for country in country_list:
    houses['mail_city'] = \
    ['FOREIGN COUNTRY' if x.__contains__(country) == True else x for x in houses['mail_city']]

### Fix misspellings of common city names
city_list = [
    'JACKSONVILLE', 'JACKSONVILLE BEACH', 'SAINT AUGUSTINE', 'SAINT AUGUSTINE BEACH', 'ATLANTIC BEACH',
    'ALEXANDRIA', 'ATLANTA', 'AMELIA ISLAND', 'BALDWIN', 'BALDWIN PARK', 'BATON ROUGE', 'BOCA RATON',
    'CINCINNATI', 'PALATKA', 'FERNANDINA', 'FERNANDINA BEACH', 'FLEMING ISLAND', 'FLAGLER BEACH', 
    'FRONT ROYAL', 'GRAND PRAIRIE', 'HIGHSTOWN', 'HOBOKEN', 'JAMAICA', 'JAMAICA ESTATES', 'JAMAICA PLAIN',
    'KEY BISCAYNE', 'LAS VEGAS', 'MANHATTAN', 'MIAMI', 'MIAMI BEACH', 'MIAMI GARDENS', 'MIAMI LAKES',
    'MIAMI SHORES', 'MIAMI SPRINGS', 'MIAMISBURG', 'MARIETTA', 'MIDDLEBURG', 'NEPTUNE BEACH', 
    'NEW ORLEANS', 'NEW SMYRNA', 'NEW SMYRNA BEACH', 'OCALA', 'PONTE VEDRA BEACH', 'PONTE VEDRA',
    'SAINT PETERSBURG', 'SAINT PETERSBURG BEACH', 'SARATOGA', 'TALLAHASSEE', 'WARNER ROBBINS', 'YULEE',
    'INDIANAPOLIS', 'LOS ANGELES', 'FORT LAUDERDALE', 'FORT COLLINS', 'LONG BEACH', 'SCOTTSDALE']

### REPLACE BURROUGHS WITH NEW YORK????

houses['mail_city'] = \
    [get_close_matches(x, city_list, n=1, cutoff=0.85)[0] if len(get_close_matches(x, city_list, \
    n=1, cutoff=0.85)) > 0 else x for x in houses['mail_city']]

<ipython-input-16-9b21e99e82fa>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  houses['mail_city'][i] = 'NEW YORK'
<ipython-input-16-9b21e99e82fa>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  houses['mail_city'][i] = 'JACKSONVILLE'


In [17]:
houses['mail_state'] = houses['mail_state'].fillna('FOREIGN COUNTRY').str.replace('FG','GA')

In [18]:
houses['mail_zipcode'] = houses['mail_zipcode'].fillna('00000').astype('str')
houses['mail_zipcode'] = [x[:5] for x in houses['mail_zipcode']]

In [19]:
houses['subdivision_name'] = houses['subdivision_name'].fillna('SECTION LAND')

In [20]:
houses['neighborhood'] = houses['neighborhood'].astype('str')

In [21]:
houses['street_num'] = houses['street_num'].fillna(0).astype('int64').astype('str')
houses['direction'] = houses['direction'].fillna('')
houses['street_name'] = houses['street_name'].fillna('NOT LISTED')
houses['street_type'] = houses['street_type'].fillna('')
houses['unit'] = houses['unit'].fillna('0')

site_address = houses['street_num'] + ' ' + houses['direction'].map(str) + ' ' + \
    houses['street_name'].map(str) + ' ' + houses['street_type'].map(str) + ' ' + houses['unit'].map(str)
site_address = [x[:-2] if x[-2:] == ' 0' else x for x in site_address]
site_address = [re.sub("\s+"," ", x) for x in site_address]
houses['site_address'] = site_address
houses.drop(columns=['street_num', 'direction', 'street_name', 'street_type', 'unit'], inplace=True)

In [22]:
houses['city'] = remove_spacing(houses['city'].fillna('JACKSONVILLE'))
houses['city'] = houses['city'].str.replace('JAX BEACH', 'JACKSONVILLE BEACH')

In [23]:
houses['zipcode'] = houses['zipcode'].fillna('00000').astype('str')
houses['zipcode'] = [x[:5] for x in houses['zipcode']]

Clean columns with other data types:

In [24]:
houses['just_value'] = houses['just_value'].astype('int64')
houses = houses[houses['just_value'] > 0]

In [25]:
houses['perc_school_deduction'] = houses['perc_school_deduction'].fillna(0)
houses['perc_school_deduction'] = houses['perc_school_deduction']/100

In [26]:
houses['perc_county_deduction'] = houses['perc_county_deduction'].fillna(0)
houses['perc_county_deduction'] = houses['perc_county_deduction']/100

In [27]:
houses['tax_district'] = houses['tax_district'].astype('category')

In [28]:
houses['lot_sf'] = houses['lot_sf'].astype('int64')

In [29]:
houses['type_descr'] = houses['type_descr'].astype('category')

In [30]:
houses['class'] = houses['class'].astype('category')

In [31]:
houses['quality'] = houses['quality'].astype('category')

In [32]:
houses['building_value'] = houses['building_value'].astype('int64')

In [33]:
houses['actual_yr_built'] = houses['actual_yr_built'].astype('int64')
houses['age'] = houses['age'].astype('int64')
houses['effec_yr_built'] = houses['effec_yr_built'].astype('int64')
houses['effec_age'] = houses['effec_age'].astype('int64')
houses['heated_sf'] = houses['heated_sf'].astype('int64')

In [34]:
subs = houses[houses.columns[houses.columns.str.startswith('sub_')==True].to_list()]
for col in subs:
    houses[col] = houses[col].fillna(0)

In [35]:
houses['land_value'] = houses['land_value'].fillna(0)
houses[houses.columns[houses.columns.str.startswith('land_')==True].to_list()].describe()

,land_value,land_use_count,land_COMM/RES/OFF,land_RES_CANAL_LD_3-7_UNITS_PER_AC,land_RES_GOLF_LD_3-7_UNITS_PER_AC,land_RES_LD_3-7_UNITS_PER_AC,land_RES_MARSH_LD_3-7_UNITS_PER_AC,land_RES_MD_8-19_UNITS_PER_AC,land_RES_NATURAL_LD_3-7_UNITS_PER_AC,land_RES_POND_LD_3-7_UNITS_PER_AC,land_RES_RIVER_LD_3-7_UNITS_PER_AC,land_RES_RURAL_2_OR_LESS_UNITS_PER_AC
count,2.527170e+05,252717.000000,252717.000000,252717.00000,252717.000000,252717.000000,252717.000000,252717.000000,252717.000000,252717.000000,252717.000000,252717.000000
mean,6.294612e+04,1.020477,0.004531,0.01252,0.008262,0.707645,0.004618,0.064064,0.057064,0.084644,0.013228,0.021906
std,1.093053e+05,0.164626,0.067158,0.11119,0.090521,0.454845,0.067798,0.244867,0.231965,0.278352,0.114251,0.146377
min,0.000000e+00,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.572500e+04,1.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,4.000000e+04,1.000000,0.000000,0.00000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,6.402700e+04,1.000000,0.000000,0.00000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,7.084800e+06,5.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [36]:
houses['feature_count'] = houses['feature_count'].fillna(0)
houses['avg_ft_grade'] = houses['avg_ft_grade'].fillna(0)
houses['features_value'] = houses['features_value'].fillna(0)

In [37]:
houses.columns

Index(['RE', 'mail_city', 'mail_state', 'mail_zipcode', 'subdivision_name',
       'neighborhood', 'perc_capped', 'just_value', 'perc_school_deduction',
       'perc_county_deduction', 'tax_district', 'lot_sf', 'building',
       'type_descr', 'class', 'quality', 'actual_yr_built', 'age',
       'effec_yr_built', 'effec_age', 'building_value', 'heated_sf',
       'ft_Carport_Al', 'ft_Cov_Patio', 'ft_Deck_Wd', 'ft_Firep_Gas',
       'ft_Firep_Ms', 'ft_Firep_Prf', 'ft_Pool', 'ft_Scr_Porch',
       'ft_Screen_En', 'ft_Shed_wood', 'bldng_count', 'land_value',
       'feature_count', 'avg_ft_grade', 'features_value', 'sub_ft_count',
       'gross_sf', 'effec_sf', 'parcel_bldng_sf', 'owner', 'baths', 'bedrooms',
       'stories', 'rooms', 'd_Canal', 'd_Golf_Course', 'd_Lake',
       'd_Local_Riverside_Avondale_Historic_District',
       'd_Local_Springfield_Historic_District', 'd_Miscellaneous_Waterway',
       'd_National_Register_Ortega_Historic_District', 'd_Ocean_Front',
       'd_St_Joh

Create adjusted just value column:

In [38]:
houses['bldng_proportion'] = round(houses['gross_sf']/houses['parcel_bldng_sf'],5)
houses['adj_just_value'] = round(houses['bldng_proportion']*houses['land_value']+houses['features_value']+\
    houses['building_value'],0).astype('int64')
houses[['adj_just_value','bldng_proportion']].head()

,adj_just_value,bldng_proportion
0,228683,1.0
1,75449,1.0
2,193391,1.0
3,95301,1.0
4,121447,1.0


In [39]:
houses.tail()

,RE,mail_city,mail_state,mail_zipcode,subdivision_name,neighborhood,perc_capped,just_value,perc_school_deduction,perc_county_deduction,...,most_recent_price,q_sales,q_sales_3yrs,RE_b,sec_township_range,care_of,mailing_address,site_address,bldng_proportion,adj_just_value
252713,181770-0100,JACKSONVILLE,FL,32224,SECTION LAND,314408.27,1.0,586022,-0.1252,-0.1679,...,NaN,NaN,NaN,181770-0100_1,38 3S 29E,None,5055 DIXIE LANDING DR,5055 DIXIE LANDING DR,1.00000,586022
252714,181771-0200,JACKSONVILLE,FL,32224,SECTION LAND,314408.27,1.0,3004603,-0.1757,-0.1840,...,NaN,NaN,NaN,181771-0200_1,38 3S 29E,None,5147 DIXIE LANDING DR,5147 DIXIE LANDING DR,1.00000,3004603
252715,181773-0000,JACKSONVILLE,FL,32224,SECTION LAND,314408.27,1.0,2117642,-0.7056,-0.7174,...,NaN,NaN,NaN,181773-0000_1,38 3S 29E,None,5187 DIXIE LANDING DR,5187 DIXIE LANDING DR,0.79542,1686952
252716,181774-0000,JACKSONVILLE,FL,32224,SECTION LAND,314408.27,1.0,1814844,-0.3788,-0.3926,...,759500.0,2.0,NaN,181774-0000_1,38 3S 29E,None,5161 DIXIE LANDING DR,5161 DIXIE LANDING DR,1.00000,1814844
252717,181776-0000,JACKSONVILLE,FL,32250,SECTION LAND,314408.26,1.0,684370,-0.2523,-0.2889,...,NaN,NaN,NaN,181776-0000_1,42 3S 29E,None,4351 PORT ARTHUR CT,4351 PORT ARTHUR CT,1.00000,684370


get_close_matches of addresses like below

In [40]:
houses.mailing_address.value_counts()[:10]

PO BOX 4090                 1239
30601 AGOURA RD 200         1093
1717 MAIN ST 2000           1052
3721 DUPONT STATION CT S     500
1508 BROOKHOLLOW DR          404
5001 PLZ ON THE LAKE 200     319
7563 PHILIPS HWY 208         293
6836 MORRISON BLVD 320       253
3721 DUPONT STATION CT       246
PO BOX 1459                  238
Name: mailing_address, dtype: int64

In [41]:
def get_top(df, col, range_end, cols):
    top_col = df[col].value_counts()[:range_end]
    top = pd.DataFrame(columns=cols)
    for num in range(range_end):
        df2 = df[df[col]==top_col.index.to_list()[num]][cols]
        for c in cols:
            df2[c] = df2[c].value_counts().index.to_list()[0]
        df2 = df2.drop_duplicates()
        top = top.append(df2)
    return top.reset_index(drop=True)


top = get_top(houses,'mailing_address',15,['mailing_address', 'mail_city', 'mail_state', 'mail_zipcode'])
top

,mailing_address,mail_city,mail_state,mail_zipcode
0,PO BOX 4090,SCOTTSDALE,AZ,85261
1,30601 AGOURA RD 200,AGOURA HILLS,CA,91301
2,1717 MAIN ST 2000,DALLAS,TX,75201
3,3721 DUPONT STATION CT S,JACKSONVILLE,FL,32217
4,1508 BROOKHOLLOW DR,SANTA ANA,CA,92705
5,5001 PLZ ON THE LAKE 200,AUSTIN,TX,78746
6,7563 PHILIPS HWY 208,JACKSONVILLE,FL,32256
7,6836 MORRISON BLVD 320,CHARLOTTE,NC,28211
8,3721 DUPONT STATION CT,JACKSONVILLE,FL,32217
9,PO BOX 1459,LA JOLLA,CA,92038


In [42]:
a = list(top['mailing_address'])
c = list(top['mail_city'])
d = dict(zip(a,c))
for address in a:
    houses.loc[houses['mailing_address']==address,'mail_city'] = d[address]

s = list(top['mail_state'])
d = dict(zip(a,s))
for address in a:
    houses.loc[houses['mailing_address']==address,'mail_state'] = d[address]
    
z = list(top['mail_zipcode'])
d = dict(zip(a,z))
for zipcode in z:
    houses.loc[houses['mailing_address']==address,'mail_zipcode'] = d[address]

In [43]:
counter = -1
for address in top['mailing_address']:
    counter += 1
    match = get_close_matches(address, top['mailing_address'].to_list()[:counter], n=1, cutoff=0.9)
    if len(match) > 0:
        top.loc[top['mailing_address']==address,'mailing_address'] = match[0]

top = top.drop_duplicates().reset_index(drop=True)

In [ ]:
for address in houses['mailing_address']:
    match = get_close_matches(address, top['mailing_address'], n=1, cutoff=0.9)
    if len(match) > 0:
        houses.loc[houses['mailing_address']==address,'mailing_address'] = match[0]



In [ ]:
houses['mailing_address'].value_counts()[:10]

In [ ]:
houses['owner_type'] = None

In [ ]:
investors = pd.DataFrame(houses['mailing_address'].value_counts())
investors = investors[investors['mailing_address'] >= 5].index.to_list()
houses.loc[houses['mailing_address'].isin(investors)==True,'owner_type'] = 'INVESTOR OWNED'
houses.loc[houses['mail_city']=='FOREIGN COUNTRY','owner_type'] = 'FOREIGN BUYER'
houses.loc[houses['mail_state']=='FOREIGN COUNTRY','owner_type'] = 'FOREIGN BUYER'
houses.loc[houses['mailing_address']==houses['site_address'],'owner_type'] = 'PRIMARY RESIDENCE'
houses['owner_type'].value_counts()

In [ ]:
ratio_list = []
houses['ratio'] = None
counter = 0
for m, s in zip(houses.loc[houses['owner_type'].isnull()==True,'mailing_address'],\
               houses.loc[houses['owner_type'].isnull()==True,'site_address']):
    counter +=1
    if counter%10000==0:
        print(counter)
#     houses.loc[(houses['owner_type'].isnull()==True)&(houses['site_address']==s)\
#                 &(houses['mailing_address']==m),'ratio'] = \
    ratio_list.append(fuzz.ratio(m,s))
houses.loc[houses['owner_type'].isnull()==True,'ratio'] = ratio_list

In [ ]:
houses.loc[houses['ratio']>=70,'owner_type'] = 'PRIMARY RESIDENCE'

In [ ]:
houses['owner_type'] = houses['owner_type'].fillna('SECONDARY RESIDENCE').astype('category')

In [ ]:
houses.drop(columns=['ratio'],inplace=True)

In [ ]:
houses.loc[houses['mailing_address']=='3500 S DUPONT HWY','mailing_address'] = '853 BROADWAY FL 5'
houses.loc[houses['mailing_address']=='3500 S DUPONT HWY','mail_city'] = 'NEW YORK'
houses.loc[houses['mailing_address']=='3500 S DUPONT HWY','mail_state'] = 'NY'
houses.loc[houses['mailing_address']=='3500 S DUPONT HWY','mail_zipcode'] = '10003'

In [ ]:
houses.to_csv('D:/Data/HousingInventory_EDA.csv')